<a href="https://colab.research.google.com/github/tamim-cloud/Gen_AI/blob/main/data_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
import pandas as pd
import numpy as np


# **Dataset link: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews**

In [10]:
data_path="/content/archive.zip"

In [11]:
df=pd.read_csv(data_path)

In [12]:
df.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire p...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue i...",positive
3,Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenl...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what mone...",positive


In [13]:
df.shape

(50000, 2)

In [14]:
df=df.iloc[:10000]

In [15]:
df.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire p...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue i...",positive
3,Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenl...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what mone...",positive


In [16]:
df.shape

(10000, 2)

In [17]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [18]:
df['sentiment'].value_counts()

,count
sentiment,
positive,5028
negative,4972


In [19]:
df.isnull().sum()

,0
review,0
sentiment,0


In [20]:
df.duplicated().sum()

np.int64(17)

In [21]:
df.drop_duplicates(inplace=True)

In [22]:
df.duplicated().sum()

np.int64(0)

# I try basic preprocessing technique <br>
**1.Remove tags-HTML <br>
  2.Lower case <br>
  3.Remove stopwords**

# 1.Remove tags-HTML

In [23]:
import re
def remove_tags(raw_text):
  cleaned_text=re.sub(re.compile('<.*?>'),'',raw_text)
  return cleaned_text

In [24]:
df['review']=df['review'].apply(remove_tags)

In [25]:
df['review'].head()

,review
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz wa..."
1,"A wonderful little production. The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. The ac..."
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue i..."
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.This movie is slower than a soap opera... and suddenly, Jake deci..."
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what mone..."


In [26]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

# 2. Lower casing

In [27]:
df['review']=df['review'].apply(lambda x:x.lower())

In [28]:
df['review'].head()

,review
0,"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.the first thing that struck me about oz wa..."
1,"a wonderful little production. the filming technique is very unassuming- very old-time-bbc fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. the ac..."
2,"i thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. the plot is simplistic, but the dialogue i..."
3,"basically there's a family where a little boy (jake) thinks there's a zombie in his closet & his parents are fighting all the time.this movie is slower than a soap opera... and suddenly, jake deci..."
4,"petter mattei's ""love in the time of money"" is a visually stunning film to watch. mr. mattei offers us a vivid portrait about human relations. this is a movie that seems to be telling us what mone..."


In [29]:
df['review'][0]

"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.i would say the main appeal of the show is due to the fact that it goes where other shows wo

#  3.Remove stopwords

In [30]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopword')
stopWordList=stopwords.words('english')
df['review']=df['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopWordList)]))

[nltk_data] Error loading stopword: Package 'stopword' not found in
[nltk_data]     index


In [31]:
df['review'].head()

,review
0,"one reviewers mentioned watching 1 oz episode hooked. right, exactly happened me.the first thing struck oz brutality unflinching scenes violence, set right word go. trust me, show faint hearted ti..."
1,"wonderful little production. filming technique unassuming- old-time-bbc fashion gives comforting, sometimes discomforting, sense realism entire piece. actors extremely well chosen- michael sheen ""..."
2,"thought wonderful way spend time hot summer weekend, sitting air conditioned theater watching light-hearted comedy. plot simplistic, dialogue witty characters likable (even well bread suspected se..."
3,"basically there's family little boy (jake) thinks there's zombie closet & parents fighting time.this movie slower soap opera... suddenly, jake decides become rambo kill zombie.ok, first going make..."
4,"petter mattei's ""love time money"" visually stunning film watch. mr. mattei offers us vivid portrait human relations. movie seems telling us money, power success people different situations encount..."


In [32]:
df['review'][0]

"one reviewers mentioned watching 1 oz episode hooked. right, exactly happened me.the first thing struck oz brutality unflinching scenes violence, set right word go. trust me, show faint hearted timid. show pulls punches regards drugs, sex violence. hardcore, classic use word.it called oz nickname given oswald maximum security state penitentary. focuses mainly emerald city, experimental section prison cells glass fronts face inwards, privacy high agenda. em city home many..aryans, muslims, gangstas, latinos, christians, italians, irish more....so scuffles, death stares, dodgy dealings shady agreements never far away.i would say main appeal show due fact goes shows dare. forget pretty pictures painted mainstream audiences, forget charm, forget romance...oz mess around. first episode ever saw struck nasty surreal, say ready it, watched more, developed taste oz, got accustomed high levels graphic violence. violence, injustice (crooked guards who'll sold nickel, inmates who'll kill order g

In [34]:
x=df.iloc[:,0:1]
y=df['sentiment']

In [35]:
x.head()

,review
0,"one reviewers mentioned watching 1 oz episode hooked. right, exactly happened me.the first thing struck oz brutality unflinching scenes violence, set right word go. trust me, show faint hearted ti..."
1,"wonderful little production. filming technique unassuming- old-time-bbc fashion gives comforting, sometimes discomforting, sense realism entire piece. actors extremely well chosen- michael sheen ""..."
2,"thought wonderful way spend time hot summer weekend, sitting air conditioned theater watching light-hearted comedy. plot simplistic, dialogue witty characters likable (even well bread suspected se..."
3,"basically there's family little boy (jake) thinks there's zombie closet & parents fighting time.this movie slower soap opera... suddenly, jake decides become rambo kill zombie.ok, first going make..."
4,"petter mattei's ""love time money"" visually stunning film watch. mr. mattei offers us vivid portrait human relations. movie seems telling us money, power success people different situations encount..."


In [36]:
y.head()

,sentiment
0,positive
1,positive
2,positive
3,negative
4,positive


# Use LabelEncoder for convert y column positive and negative into 1 and 0 number

In [37]:
from sklearn.preprocessing import LabelEncoder

In [38]:
encoder=LabelEncoder()

In [39]:
y=encoder.fit_transform(y)

In [40]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [41]:
from sklearn.model_selection import train_test_split

In [64]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

In [65]:
x_train.shape

(7986, 1)

In [66]:
x_test.shape

(1997, 1)

# Now apply some Feature Engineering <br>
**1. Bag of words <br> 2. RandomForestclassifier <br> 3. N-grams(Bi-gram) <br> 4. TF-IDF <br> 5. Wordvectorization**

# Bag of words

In [67]:
from sklearn.feature_extraction.text import CountVectorizer

In [68]:
cv=CountVectorizer()

In [69]:
x_train_bow=cv.fit_transform(x_train['review']).toarray()

In [70]:
x_test_bow=cv.transform(x_test['review']).toarray()

In [71]:
x_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [72]:
x_test_bow


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# 1. GaussianNB (probabilistic classifier)

In [96]:
from sklearn.naive_bayes import GaussianNB # for remove extra 0 from matrix we use gaussian naive bayes algorithm, naive bayes good for classification model

In [97]:
gnb=GaussianNB()

In [98]:
gnb.fit(x_train_bow,y_train)

GaussianNB()

In [99]:
y_pred=gnb.predict(x_test_bow)

In [100]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [101]:
accuracy_score(y_test,y_pred)

0.7681522283425137

In [102]:
confusion_matrix(y_test,y_pred)

array([[796, 156],
       [307, 738]])

# 2. Applying RandomForestclassifier

In [1]:
from sklearn.ensemble import RandomForestClassifier #most of the case it performs good

In [2]:
rf=RandomForestClassifier()

In [51]:
rf.fit(x_train_bow,y_train)

RandomForestClassifier()

In [93]:
y_pred=rf.predict(x_test_bow)

array([1, 1, 0, ..., 1, 0, 0])

In [94]:
accuracy_score(y_pred,y_test)

0.8457686529794692

# Again CounterVectorizer with 5000 data

In [81]:
cv=CountVectorizer(max_features=5000)

In [82]:
x_train_bow=cv.fit_transform(x_train['review']).toarray()

In [83]:
x_test_bow=cv.transform(x_test['review']).toarray()

In [84]:
rf=RandomForestClassifier()

In [85]:
rf.fit(x_train_bow,y_train)

RandomForestClassifier()

In [86]:
y_pred=rf.predict(x_test_bow)

In [87]:
accuracy_score(y_test,y_pred)

0.8457686529794692

In [89]:
from sklearn.naive_bayes import GaussianNB

In [90]:
gnb=GaussianNB()

In [91]:
gnb.fit(x_train_bow,y_train)

GaussianNB()

In [92]:
y_pred=gnb.predict(x_test_bow)

In [95]:
accuracy_score(y_test,y_pred)

0.8457686529794692

# 2. N-grams (Bi-grams)

In [110]:
cv=CountVectorizer(ngram_range=(1,2),max_features=5000)

In [111]:
x_train_ngram=cv.fit_transform(x_train['review']).toarray()

In [112]:
x_test_ngram=cv.transform(x_test['review']).toarray()

In [113]:
rf=RandomForestClassifier()

In [114]:
rf.fit(x_train_ngram,y_train)

RandomForestClassifier()

In [118]:
y_pred=rf.predict(x_test_ngram)

In [119]:
accuracy_score(y_test,y_pred)

0.8432648973460191

In [120]:
gnb_ngram=GaussianNB()

In [121]:
gnb_ngram.fit(x_train_ngram,y_train)

GaussianNB()

In [123]:
y_pred_ngram_gaussian=gnb_ngram.predict(x_test_ngram)

In [125]:
accuracy_score(y_test,y_pred_ngram_gaussian)

0.8077115673510266

# 5. TF-IDF

In [127]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [128]:
tfidf=TfidfVectorizer()

In [129]:
x_train_tfidf=tfidf.fit_transform(x_train['review']).toarray()

In [138]:
x_test_tfidf=tfidf.transform(x_test['review']).toarray()

**Random Forest Clasification**

In [139]:
rf=RandomForestClassifier()

In [140]:
rf.fit(x_train_tfidf,y_train)

RandomForestClassifier()

In [141]:
y_pred=rf.predict(x_test_tfidf)

In [142]:
accuracy_score(y_test,y_pred)

0.8487731597396094

**Naive Bayes Gaussian**

In [149]:
gnb_tfidf=GaussianNB()

In [150]:
gnb_tfidf.fit(x_train_tfidf,y_train)

GaussianNB()

In [151]:
y_pred_tfidf_gaussian=gnb_tfidf.predict(x_test_tfidf)

In [152]:
accuracy_score(y_pred_tfidf_gaussian,y_test)

0.6389584376564847